In [0]:

from pyspark.sql.functions import col, current_date, year, month, dayofmonth, when, date_format, to_date, regexp_replace, regexp_extract, sum, countDistinct
from delta.tables import *

# Leer tablas silver

#Dataframes
appearances_silver = (
    spark
    .table("football.`silver-football-data`.appearances_silver"))
club_games_silver = (
    spark
    .table("football.`silver-football-data`.club_games_silver"))
clubs_silver = (
    spark
    .table("football.`silver-football-data`.clubs_silver"))
game_events_silver = (
    spark
    .table("football.`silver-football-data`.game_events_silver"))
game_lineups_silver = (
    spark
    .table("football.`silver-football-data`.game_lineups_silver"))
games_silver = (
    spark
    .table("football.`silver-football-data`.games_silver"))
player_valuations_silver = (
    spark
    .table("football.`silver-football-data`.player_valuations_silver"))
players_silver = (
    spark
    .table("football.`silver-football-data`.players_silver"))
transfers_silver = (
    spark
    .table("football.`silver-football-data`.transfers_silver"))
competitions_silver = (
    spark
    .table("football.`silver-football-data`.competitions_silver"))

#Delta Tables
competitions_silver_dt = (
    DeltaTable
    .forName(spark,"football.`silver-football-data`.competitions_silver"))

#TABLA appearances 
appearances_gold = (
    appearances_silver
    #Añadir columna season
    .join(
        games_silver
            .select("game_id", "season"),
        on="game_id",
        how="left"
    ) 
    .select("player_id", "goals","game_id", "assists", "minutes_played","season", "player_name")
    #Agrupar por jugador y temporada y sumar los valores de goals, assists y minutes_played
    .groupBy("player_id", "player_name", "season")
    .agg(
        sum("goals").alias("goals"),
        sum("assists").alias("assists"),
        sum("minutes_played").alias("minutes_played"),
        countDistinct("game_id").alias("games_played")
    )
)

#TABLA club_games
club_games_gold = (
    club_games_silver
    #Añadir columna season, competition_id y date
    .join(
        games_silver
            .select("game_id", "season", "competition_id", "date"),
        on="game_id",
        how="left"
    )
    .join(
        competitions_silver
        .select("competition_id", col("name").alias("competition_name")),
        on="competition_id",
        how="left"
    )
    .select("game_id", "club_id","hosting","is_win", "season", "competition_id", "competition_name", "date")
)

#TABLA clubs
clubs_gold = (
    clubs_silver
    #Añadir columna competition_name
    .join(
        competitions_silver
          .select("competition_id", col("name").alias("competition_name")),
        on=col("domestic_competition_id") == col("competition_id"),
        how="left"
    )   
)

#TABLA competitions
competitions_silver_dt.clone(
    target="football.`gold-football-data`.competitions_gold",isShallow=True, replace=True)

#TABLA game_events
game_events_gold = (
    game_events_silver
    #Eliminar columnas no utilizadas
    .drop("date", "player_id", "description", "player_in_id", "player_assist_id")
)

#TABLA game_lineups
game_lineups_gold = (
    #Añadir columna club_name
    game_lineups_silver
    .join(
        clubs_silver
            .select("club_id", "name"),
        on="club_id",
        how="left"
    )
    .withColumnRenamed("name", "club_name")
    .drop("date")
)

# LIMPIAR TABLA games
games_gold = (
    games_silver
    .join(
        competitions_silver
            .select("competition_id", "is_major_national_league", col("name").alias("competition_name")),
        on="competition_id",
        how="left"
    )
    
)

#TABLA players_valuation
player_valuations_gold = (
    player_valuations_silver
    .join(
        players_silver
            .select("player_id", "name"),
        on="player_id"
    )
    .withColumnRenamed("name", "player_name")
    .withColumn(
        "season",
        when(month(col("date")) >= 8, year(col("date")))
        .otherwise(year(col("date")) - 1)
    )
    .drop("current_club_id", "player_club_domestic_competition_id")
)

#TABLA players
players_gold = (
    players_silver
    .join(
          competitions_silver
          .select("competition_id", col("name").alias("competition_name")),
        on=col("current_club_domestic_competition_id") == col("competition_id"),
        how="left")
    #Añadir columna age
    .join(
        spark.table("football.`gold-football-data`.players_age_view").select("player_id", "age"),
        on="player_id",
        how="left"
    )
    .drop("player_code","agent_name")
)

#TABLA transfers
transfers_gold = (
    transfers_silver
    .drop("market_value_in_eur")
)


# Guardar como tablas gold
(appearances_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.appearances_gold"))
(club_games_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.club_games_gold"))
(clubs_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.clubs_gold"))
(game_events_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.game_events_gold"))
(game_lineups_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.game_lineups_gold"))
(games_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.games_gold"))
(player_valuations_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.player_valuations_gold"))
(players_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.players_gold"))
(transfers_gold
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`gold-football-data`.transfers_gold"))